In [74]:
#importing libraries

import cv2
import os
import numpy as np
import pandas as pd
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
import import_ipynb
from imagePreProcessor import *
from TesseractModify import *
from datetime import datetime
import math
import dateparser
import datefinder
from IPython.display import clear_output
import difflib

In [75]:
imagesFolderPath = '/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Data/Images'

In [76]:
def preProcessDataFrame(df, image):
    
    #drop null values
    df.dropna(inplace=True)
    
    
    #rows having text as a stopword removed
    indexesToDrop = []
    stopwords = ["", " ", "  ", "/", ":", "-", ".", ",", "\n", "\t", "\\", "(", ")", "[", "]", "{", "}", "*", "&", "%", "$", "#", "|"]
    for index in df.index:
        if df["text"][index] in stopwords:
            indexesToDrop.append(index)   
    df.drop(indexesToDrop, inplace=True)
    
    #resetting the index
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)

    for index in df.index:
        df["text"][index] = df["text"][index].lower()
    
    #creating new columns for centroid information
    df["x"] = df["left"] + df["width"] 
    df["y"] = df["top"] + df["height"]
    
    #new columns for page width and height for further normalization
    df["PageHeight"] = image.shape[0]
    df["PageWidth"] = image.shape[1]
    
    #adding columns for zonal info (4 Zone)
    df["isTop"] = 0
    df["isBottom"] = 0
    df["isRight"] = 0
    df["isLeft"] = 0

In [77]:
def verifyDate(text):
    
    dates = list(datefinder.find_dates(text, strict=True))
    
    if len(dates)==0:
        return False
    else:
        for date in dates:
            year = date.year
            presentYear = date.today().year
            if abs(year-presentYear)<=2:
                print(dates)
                return True
            else:
                return False

In [78]:
def findDate(dateDF):
    dates = []
    
    for index in dateDF.index:
        text = dateDF["text"][index]
        if verifyDate(text):
            dates.append(text)
    return dates

In [79]:
def extractDateDataFrame(df):
    dateDF = df.copy()
    for index in dateDF.index:
        text = dateDF["text"][index]
        dateDF["text"][index] = text.lower()
    dates = findDate(dateDF)
    return dateDF, dates
    
        

In [80]:
def returnDist(x1, y1, x2, y2):
    p = [x1, y1]
    q = [x2, y2]
    return math.dist(p, q)

In [81]:
def compare(word, neighbours):
    if word in neighbours:
        return word
    else:
        matches = difflib.get_close_matches(word, neighbours, n=1, cutoff=0.8)
        if len(matches)==0:
            return None
        else:
            return matches[0]
            

In [148]:
def allocateNeighbours(df, neighbourFields, neighbourFieldsX, neighbourFieldsY):
    for index in df.index:
        x1 = df["x"][index]
        y1 = df["y"][index]
        block = df["block_num"][index]
        neighbours = {}
        pw = df["PageWidth"][index]
        ph = df["PageHeight"][index]
        pd = np.sqrt((pw**2) + (ph**2))
        distFact = pd*0.3
        for index2 in df.index:
            x2 = df["x"][index2]
            y2 = df["y"][index2]
            block2 = df["block_num"][index2]
            dist = returnDist(x1, y1, x2, y2)
            if x2<=x1 and y2<=y1:
                if dist<=distFact:
                    neighbours[df["text"][index2].lower()]=[x2, y2]
                elif abs(block2-block)<=1:
                    neighbours[df["text"][index2].lower()]=[x2, y2]
                    
        for i in range(len(neighbourFields)):
            n = neighbourFields[i]
            nx = neighbourFieldsX[i]
            ny = neighbourFieldsY[i]
            if compare(n, neighbours.keys())==None:
                pass
            else:
                df[n][index]=1
                coord = neighbours[compare(n, neighbours.keys())]
                df[nx][index]=coord[0]
                df[ny][index]=coord[1]




In [149]:
def addNeighbours(df):
    neighbourFields = ["invoice date", "invoice dt", "due date", "payment date", "ack date", "ack dt", "bill dt", "receipt date", "receipt dt"]
    
    neighbourFieldsX = ["invoice date x", "invoice dt x", "due date x", "payment date x", "ack date x", "ack dt x", "bill dt x", "receipt date x", "receipt dt x"]

    neighbourFieldsY = ["invoice date y", "invoice dt y", "due date y", "payment date y", "ack date y", "ack dt y", "bill dt y", "receipt date y", "receipt dt y"]

    for col in neighbourFields:
        df[col]=0
    for col in neighbourFieldsX:
        df[col]=0
    for col in neighbourFieldsY:
        df[col]=0
    allocateNeighbours(df, neighbourFields, neighbourFieldsX, neighbourFieldsY)
    

In [150]:
def findDateDF(dates, dateDF):
    indexes = []
    for date in dates:
        i = dateDF[dateDF["text"]==date].index
        for index in i:
            indexes.append(index)
    return indexes

In [151]:
def dropIndexes(indexes, df):
    indexesToDrop=[]
    for index in df.index:
        if index not in indexes:
            indexesToDrop.append(index)
    df.drop(indexesToDrop, inplace=True)

In [152]:
def create(imagePath):
    image = cv2.imread(imagePath)
    preProcessedImage = preProcessImage(imagePath)
    data = pytesseract.image_to_data(preProcessedImage, output_type=Output.DATAFRAME)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    data.drop(columns=["index"], inplace=True)
    processData(data)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, preProcessedImage)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    return resultDF
    
    

In [153]:
def batchCreate(folderPath):
    os.chdir(folderPath)
    images = os.listdir()
    dfList = []
    for imagePath in images:
        #print(imagePath)
        df = create(imagePath)
        df["imageName"] = imagePath
        df["output"] = 0
        dfList.append(df)
    return dfList

In [154]:
os.chdir(imagesFolderPath)

In [161]:
imagePath = "12.jpeg"

In [162]:
df = create(imagePath)
df

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5540/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_5540/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()


[datetime.datetime(2023, 1, 31, 0, 0)]
[datetime.datetime(2023, 10, 2, 0, 0)]
[datetime.datetime(2023, 1, 31, 0, 0)]
[datetime.datetime(2023, 1, 31, 0, 0)]
[datetime.datetime(2023, 10, 2, 0, 0)]
[datetime.datetime(2023, 1, 31, 12, 57)]


/tmp/ipykernel_5540/825851322.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_5540/825851322.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nx][index]=coord[0]
/tmp/ipykernel_5540/825851322.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ny][index]=coord[1]


,block_num,line_num,left,top,width,height,conf,text,nextDist,x,...,receipt dt x,invoice date y,invoice dt y,due date y,payment date y,ack date y,ack dt y,bill dt y,receipt date y,receipt dt y
32,2,7,716,561,193,39,0.000000,31/01/2023,373,909,...,0,595,595,0,0,0,0,0,0,0
47,2,9,716,653,193,33,0.000000,$10/02/2023,373,909,...,0,595,595,680,0,0,0,0,0,0
235,12,2,933,2497,170,30,96.451363,2023-01-31,13,1103,...,0,0,1959,0,1697,2526,0,0,0,0
269,2,0,716,561,193,39,0.000000,31/01/2023,373,909,...,0,595,595,0,0,0,0,0,0,0
277,2,0,716,653,193,33,0.000000,$10/02/2023,373,909,...,0,595,595,680,0,0,0,0,0,0
361,12,0,933,2497,297,31,96.655850,2023-01-31 12:57:00,-598,1230,...,0,0,1959,0,1697,2526,0,0,0,0


In [164]:
for col in ["invoice date x", "invoice dt x", "due date x", "payment date x", "ack date x", "ack dt x", "bill dt x", "receipt date x", "receipt dt x"]:
    print(col, df[col][32])


invoice date x 358
invoice dt x 358
due date x 0
payment date x 0
ack date x 0
ack dt x 0
bill dt x 0
receipt date x 0
receipt dt x 0


In [165]:
dfList = batchCreate(imagesFolderPath)

<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5540/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_5540/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_5540/825851322.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5540/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_5540/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()


[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]
[datetime.datetime(2022, 5, 12, 0, 0)]


/tmp/ipykernel_5540/825851322.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_5540/825851322.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[nx][index]=coord[0]
/tmp/ipykernel_5540/825851322.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ny][index]=coord[1]
<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

...

[datetime.datetime(2022, 12, 24, 0, 0)]
[datetime.datetime(2022, 12, 27, 0, 0)]
[datetime.datetime(2022, 12, 24, 0, 0)]
[datetime.datetime(2022, 12, 27, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5540/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_5540/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_5540/825851322.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

[datetime.datetime(2023, 2, 14, 0, 0)]
[datetime.datetime(2023, 2, 14, 0, 0)]


<string>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_5540/1412637721.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][index] = df["text"][index].lower()
/tmp/ipykernel_5540/3296310189.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = text.lower()
/tmp/ipykernel_5540/825851322.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

[datetime.datetime(2022, 9, 28, 0, 0)]
[datetime.datetime(2022, 9, 28, 0, 0)]


In [166]:
len(dfList)

110

In [191]:
resultDF = pd.concat(dfList)

In [192]:
resultDF.describe()

,block_num,line_num,left,top,width,height,conf,nextDist,x,y,...,invoice date y,invoice dt y,due date y,payment date y,ack date y,ack dt y,bill dt y,receipt date y,receipt dt y,output
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,...,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0
mean,9.648829,1.632107,1235.505017,1136.431438,190.010033,32.411371,82.248156,-684.695652,1425.515050,1168.842809,...,234.702341,400.384615,38.006689,29.692308,37.551839,28.107023,46.548495,5.591973,23.361204,0.0
std,8.896521,3.393406,752.495037,738.282392,75.580371,7.629948,20.776512,954.989468,751.611335,738.461133,...,421.177777,588.214249,172.992318,189.934232,295.913895,171.614526,173.502375,68.258348,201.404673,0.0
min,1.000000,0.000000,75.000000,67.000000,98.000000,16.000000,0.000000,-3189.000000,210.000000,97.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,4.000000,0.000000,556.000000,598.500000,154.000000,28.000000,74.454445,-1627.000000,783.500000,632.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,6.000000,1.000000,1367.000000,915.000000,178.000000,32.000000,90.819194,-401.000000,1595.000000,937.000000,...,0.000000,120.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,12.000000,2.000000,1908.000000,1515.500000,203.000000,37.000000,96.216804,118.000000,2113.000000,1554.500000,...,495.000000,599.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,46.000000,26.000000,3130.000000,3395.000000,821.000000,71.000000,96.945030,2035.000000,3278.000000,3424.000000,...,2995.000000,2995.000000,1336.000000,1697.000000,2571.000000,1717.000000,949.000000,836.000000,2312.000000,0.0


In [193]:
len(resultDF["imageName"].unique())

72

In [194]:
resultDF.reset_index(inplace=True)

In [195]:
resultDF.drop(columns=["index", "nextDist", "block_num", "line_num"], inplace=True)

In [196]:
resultDF.shape

(299, 43)

In [197]:
resultDF.head()

,left,top,width,height,conf,text,x,y,PageHeight,PageWidth,...,invoice dt y,due date y,payment date y,ack date y,ack dt y,bill dt y,receipt date y,receipt dt y,imageName,output
0,552,1142,269,34,46.287964,dot::05/12/2022,821,1176,3500,2525,...,0,0,0,0,0,0,0,0,20.jpeg,0
1,552,1142,269,34,46.287964,dot::05/12/2022,821,1176,3500,2525,...,0,0,0,0,0,0,0,0,20.jpeg,0
2,2005,536,179,28,96.255013,05/12/2022,2184,564,3300,2550,...,502,0,0,0,0,0,0,0,23.jpeg,0
3,2005,681,184,29,96.816185,05/12/2022,2189,710,3300,2550,...,502,0,0,0,0,0,0,0,23.jpeg,0
4,1820,1355,184,29,94.628899,05/12/2022,2004,1384,3300,2550,...,502,0,0,0,0,0,0,0,23.jpeg,0


In [198]:
resultDF.describe()

,left,top,width,height,conf,x,y,PageHeight,PageWidth,isTop,...,invoice date y,invoice dt y,due date y,payment date y,ack date y,ack dt y,bill dt y,receipt date y,receipt dt y,output
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0,...,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0
mean,1235.505017,1136.431438,190.010033,32.411371,82.248156,1425.515050,1168.842809,3459.036789,2572.792642,0.0,...,234.702341,400.384615,38.006689,29.692308,37.551839,28.107023,46.548495,5.591973,23.361204,0.0
std,752.495037,738.282392,75.580371,7.629948,20.776512,751.611335,738.461133,464.527813,322.096359,0.0,...,421.177777,588.214249,172.992318,189.934232,295.913895,171.614526,173.502375,68.258348,201.404673,0.0
min,75.000000,67.000000,98.000000,16.000000,0.000000,210.000000,97.000000,1566.000000,1134.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,556.000000,598.500000,154.000000,28.000000,74.454445,783.500000,632.000000,3300.000000,2479.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,1367.000000,915.000000,178.000000,32.000000,90.819194,1595.000000,937.000000,3484.000000,2481.000000,0.0,...,0.000000,120.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,1908.000000,1515.500000,203.000000,37.000000,96.216804,2113.000000,1554.500000,3508.000000,2550.000000,0.0,...,495.000000,599.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,3130.000000,3395.000000,821.000000,71.000000,96.945030,3278.000000,3424.000000,5313.000000,3542.000000,0.0,...,2995.000000,2995.000000,1336.000000,1697.000000,2571.000000,1717.000000,949.000000,836.000000,2312.000000,0.0


In [199]:
# resultDF["contextPresent"]=0

In [200]:
# contexts = ["invoice date", "invoice", "invoice dt", "dated", "date", "bill dt", "receipt date", "receipt dt", "bill", "receipt"]

In [201]:
# for index in resultDF.index:
#     cont = []
#     for col in contexts:
#         cont.append(resultDF[col][index])
#     if max(cont)>0:
#         resultDF["contextPresent"][index]=1

In [202]:
# resultDF.describe()

In [203]:
# resultDF["contextDist"] = 0

In [204]:
# for index in resultDF.index:
#     if resultDF["contextPresent"][index]==1:
#         cont = []
#         for col in contexts:
#             if resultDF[col][index]>0:
#                 cont.append(resultDF[col][index])
#         dist = min(cont)
#         resultDF["contextDist"][index] = dist


In [205]:
# resultDF.drop(columns=["output"], inplace=True)

In [206]:
# resultDF["output"] = 0

In [207]:
resultDF

,left,top,width,height,conf,text,x,y,PageHeight,PageWidth,...,invoice dt y,due date y,payment date y,ack date y,ack dt y,bill dt y,receipt date y,receipt dt y,imageName,output
0,552,1142,269,34,46.287964,dot::05/12/2022,821,1176,3500,2525,...,0,0,0,0,0,0,0,0,20.jpeg,0
1,552,1142,269,34,46.287964,dot::05/12/2022,821,1176,3500,2525,...,0,0,0,0,0,0,0,0,20.jpeg,0
2,2005,536,179,28,96.255013,05/12/2022,2184,564,3300,2550,...,502,0,0,0,0,0,0,0,23.jpeg,0
3,2005,681,184,29,96.816185,05/12/2022,2189,710,3300,2550,...,502,0,0,0,0,0,0,0,23.jpeg,0
4,1820,1355,184,29,94.628899,05/12/2022,2004,1384,3300,2550,...,502,0,0,0,0,0,0,0,23.jpeg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,2006,1630,171,26,82.106323,27-dec-22,2177,1656,3508,2479,...,1593,0,0,0,0,0,0,0,86.jpeg,0
295,1704,225,181,31,90.515984,14-feb-23,1885,256,3300,2550,...,112,0,0,0,0,0,0,0,46.jpeg,0
296,1704,225,181,31,90.515984,14-feb-23,1885,256,3300,2550,...,112,0,0,0,0,0,0,0,46.jpeg,0
297,1696,208,194,42,96.214699,28-sep-22,1890,250,3508,2479,...,142,0,0,0,0,0,0,0,62.jpeg,0


In [208]:
for index in resultDF.index:
    name = resultDF["imageName"][index].split(".")[0]
    resultDF["imageName"][index]=int(name)

/tmp/ipykernel_5540/2442472026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultDF["imageName"][index]=int(name)


In [209]:
resultDF.sort_values("imageName", inplace=True)
resultDF

,left,top,width,height,conf,text,x,y,PageHeight,PageWidth,...,invoice dt y,due date y,payment date y,ack date y,ack dt y,bill dt y,receipt date y,receipt dt y,imageName,output
72,1713,256,203,28,96.087402,21-07-2022,1916,284,3504,2479,...,136,0,0,0,0,0,0,0,1,0
73,1713,256,203,28,96.087402,21-07-2022,1916,284,3504,2479,...,136,0,0,0,0,0,0,0,1,0
206,1648,1936,214,31,79.921706,1-oct-22. to 31,1862,1967,3504,2479,...,976,0,0,0,0,0,0,0,2,0
204,1648,1936,136,31,48.239994,1-oct-22.,1784,1967,3504,2479,...,976,0,0,0,0,0,0,0,2,0
203,376,486,163,35,95.528168,22-sep-22,539,521,3504,2479,...,0,0,0,517,517,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,2133,1209,185,30,95.900703,01-feb-2023,2318,1239,3300,2550,...,0,0,0,0,0,0,0,0,97,0
270,2098,536,159,28,96.263855,21-08-2022,2257,564,3508,2479,...,0,0,0,0,0,0,0,175,98,0
88,742,508,98,25,57.750854,07-feb-2023,840,533,1566,1134,...,0,0,0,0,0,0,0,0,100,0
143,1669,449,243,49,96.855301,01/10/2022,1912,498,3507,2480,...,498,0,0,0,0,0,0,0,107,0


In [210]:
# for index in resultDF.index:
#     name = str(resultDF["imageName"][index]) + ".jpeg"
#     resultDF["imageName"][index]=name
# resultDF

In [211]:
os.chdir('/home/aman/Documents/Tally/Git-Document-AI/Document-AI/InvoiceDateModel-v2/Dataset/')

In [212]:
pwd

'/home/aman/Documents/Tally/Git-Document-AI/Document-AI/InvoiceDateModel-v2/Dataset'

In [213]:
resultDF.to_csv("tesseract4.csv")